In [7]:
from langchain_google_vertexai import ChatVertexAI
from langchain_core.prompts import PromptTemplate

llm = ChatVertexAI(model="gemini-2.0-flash-001")

# Custom tools

We will use `numexpr` library to evaluate math expressions:

In [2]:
import numexpr as ne
import math

math_constants = {"pi": math.pi, "i": 1j, "e": math.exp},
c = ne.evaluate(("2+2"), local_dict=math_constants)


In [3]:
math_constants = {"pi": math.pi, "i": 1j, "e": math.exp}
ne.evaluate("(2+3*i)**2", local_dict=math_constants)

array(-5.+12.j)

We define our calculator as a Python function and wrap it with a built-in `@tool` decorator to create a tool from it:

In [4]:
from langchain_core.tools import tool


@tool
def calculator(expression: str) -> str:
    """Calculates a single mathematical expression, incl. complex numbers.

    Always add * to operations, examples:
      73i -> 73*i
      7pi**2 -> 7*pi**2
    """
    math_constants = {"pi": math.pi, "i": 1j, "e": math.exp}
    result = ne.evaluate(expression.strip(), local_dict=math_constants)
    return str(result)


In [5]:
from langchain_core.tools import BaseTool

assert isinstance(calculator, BaseTool)
print(f"Tool name: {calculator.name}")
print(f"Tool name: {calculator.description}")
print(f"Tool schema: {calculator.args_schema.model_json_schema()}")

Tool name: calculator
Tool name: Calculates a single mathematical expression, incl. complex numbers.

    Always add * to operations, examples:
      73i -> 73*i
      7pi**2 -> 7*pi**2
Tool schema: {'description': 'Calculates a single mathematical expression, incl. complex numbers.\n\nAlways add * to operations, examples:\n  73i -> 73*i\n  7pi**2 -> 7*pi**2', 'properties': {'expression': {'title': 'Expression', 'type': 'string'}}, 'required': ['expression'], 'title': 'calculator', 'type': 'object'}


In [8]:
from langgraph.prebuilt import create_react_agent

query = "How much is 2+3i squared?"

agent = create_react_agent(llm, [calculator])

for event in agent.stream({"messages": [("user", query)]}, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

How much is 2+3i squared?
================================== Ai Message ==================================
Tool Calls:
  calculator (799f8b33-03cf-4920-bdbe-52680662fb5f)
 Call ID: 799f8b33-03cf-4920-bdbe-52680662fb5f
  Args:
    expression: (2+3*i)**2
================================= Tool Message =================================
Name: calculator

(-5+12j)
================================== Ai Message ==================================

(2+3i)^2 is -5+12j.


We will re-use the `search` tool we created in the previous section:

In [9]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

In [11]:
question = "What is a square root of the current US president’s age multiplied by 132?"

system_hint = "Think step-by-step. Always use search to get the fresh information about events or public facts that can change over time. Now is 2025 and remember president elections in the US recently happened."

agent = create_react_agent(
    llm, [calculator, search],
    state_modifier=system_hint)

for event in agent.stream({"messages": [("user", question)]}, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is a square root of the current US president’s age multiplied by 132?
================================== Ai Message ==================================
Tool Calls:
  duckduckgo_search (364bee40-efd2-4f08-96c6-84f2bd1f4ca0)
 Call ID: 364bee40-efd2-4f08-96c6-84f2bd1f4ca0
  Args:
    query: current US president age
================================= Tool Message =================================
Name: duckduckgo_search

Following Jimmy Carter's death at the age of 100, there are five living former or current U.S. presidents. At 82 years old, Joe Biden is the eldest of the group. He's closely followed by ... The White House, official residence of the president of the United States, in July 2008. The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via the Electoral College. [2] Under the U.S. Constitution, the office

We can explore the final answer the model produced:

In [12]:
print(event["messages"][-1].content)

The square root of the current US president's age multiplied by 132 is approximately 101.47.



We can also create a tool from a `Runnable` (and again, we can convery any Python function to a `Runnable` by using `RunnableLambda`):

In [13]:
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import convert_runnable_to_tool


def calculator(expression: str) -> str:
    math_constants = {"pi": math.pi, "i": 1j, "e": math.exp}
    result = ne.evaluate(expression.strip(), local_dict=math_constants)
    return str(result)


calculator_with_retry = RunnableLambda(calculator).with_retry(
    wait_exponential_jitter=True,
    stop_after_attempt=3,
)


calculator_tool = convert_runnable_to_tool(
    calculator_with_retry,
    name="calculator",
    description=(
        "Calculates a single mathematical expression, incl. complex numbers."
        "'\nAlways add * to operations, examples:\n73i -> 73*i\n"
        "7pi**2 -> 7*pi**2"
    ),
    arg_types={"expression": "str"},
)

In [15]:
calculator_tool.invoke({"expression": "(2+3*i)**2"})

'(-5+12j)'

In [16]:
llm.invoke("How much is (2+3i)**2", tools=[calculator_tool]).tool_calls[0]

{'name': 'calculator',
 'args': {'__arg1': '(2+3*i)**2'},
 'id': '3ca39b86-695b-4b21-a058-5d564e705743',
 'type': 'tool_call'}

We can also pass a custom arguments schema when we create a tool with `convert_runnable_to_tool`:

In [18]:
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableConfig


class CalculatorArgs(BaseModel):
    expression: str = Field(description="Mathematical expression to be evaluated")


def calculator(state: CalculatorArgs, config: RunnableConfig) -> str:
    expression = state["expression"]
    math_constants = config["configurable"].get("math_constants", {})
    result = ne.evaluate(expression.strip(), local_dict=math_constants)
    return str(result)


calculator_with_retry = RunnableLambda(calculator).with_retry(
    wait_exponential_jitter=True,
    stop_after_attempt=3,
)

calculator_tool = convert_runnable_to_tool(
    calculator_with_retry,
    name="calculator",
    description=(
        "Calculates a single mathematical expression, incl. complex numbers."
        "'\nAlways add * to operations, examples:\n73i -> 73*i\n"
        "7pi**2 -> 7*pi**2"
    ),
    args_schema=CalculatorArgs,
    arg_types={"expression": "str"},
)

In [19]:
assert isinstance(calculator_tool, BaseTool)
print(f"Tool name: {calculator_tool.name}")
print(f"Tool name: {calculator_tool.description}")
print(f"Args schema: {calculator_tool.args_schema.model_json_schema()}")

Tool name: calculator
Tool name: Calculates a single mathematical expression, incl. complex numbers.'
Always add * to operations, examples:
73i -> 73*i
7pi**2 -> 7*pi**2
Args schema: {'properties': {'expression': {'title': 'Expression', 'type': 'string'}}, 'required': ['expression'], 'title': 'calculator', 'type': 'object'}


That's how we pass configuration to our new tool:

In [20]:
math_constants = {"pi": math.pi, "i": 1j, "e": math.exp}
config = {"configurable": {"math_constants": math_constants}}

tool_call = llm.invoke("How much is (2+3i)**2", tools=[calculator_tool]).tool_calls[0]
print(tool_call)

{'name': 'calculator', 'args': {'expression': '(2+3*i)**2'}, 'id': '1d032f30-8e22-4f4d-885f-174b5c23a7bc', 'type': 'tool_call'}


In [21]:
calculator_tool.invoke(tool_call["args"], config=config)

'(-5+12j)'

In [23]:
from langchain_core.tools import StructuredTool

def calculator(expression: str) -> str:
    math_constants = {"pi": math.pi, "i": 1j, "e": math.exp}
    result = ne.evaluate(expression.strip(), local_dict=math_constants)
    return str(result)

calculator_tool = StructuredTool.from_function(
    name="calculator",
    description=(
        "Calculates a single mathematical expression, incl. complex numbers."),
    func=calculator,
    args_schema=CalculatorArgs
)

tool_call = llm.invoke("How much is (2+3i)**2", tools=[calculator_tool]).tool_calls[0]
print(tool_call)

{'name': 'calculator', 'args': {'expression': '(2+3i)**2'}, 'id': '5e48037b-5ae8-4b5d-832e-aef8a2ee5fd1', 'type': 'tool_call'}


We can also take a look at how our agents adapts to the feedback from the environment and how an LLM corrects the input sent to the tool:

In [27]:
from langchain_core.tools import StructuredTool

def calculator(expression: str) -> str:
    """Calculates a single mathematical expression, incl. complex numbers."""
    return str(ne.evaluate(expression.strip(), local_dict={}))

calculator_tool = StructuredTool.from_function(
    func=calculator,
    handle_tool_error=True
)

agent = create_react_agent(
    llm, [calculator_tool])

for event in agent.stream({"messages": [("user", "How much is (2+3i)^2")]}, stream_mode="values", config=config):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

How much is (2+3i)^2
================================== Ai Message ==================================
Tool Calls:
  calculator (98fc1fae-d5a6-4d32-9fe9-b3017e5e8bf8)
 Call ID: 98fc1fae-d5a6-4d32-9fe9-b3017e5e8bf8
  Args:
    expression: (2+3i)^2
================================= Tool Message =================================
Name: calculator

Error: SyntaxError('invalid decimal literal', ('<expr>', 1, 4, '(2+3i)^2', 1, 4))
 Please fix your mistakes.
================================== Ai Message ==================================

I am sorry, I encountered an error. I will try again.
Tool Calls:
  calculator (1332e6bc-d6a3-4649-8992-f7bb8da77d11)
 Call ID: 1332e6bc-d6a3-4649-8992-f7bb8da77d11
  Args:
    expression: (2+3*i)^2
================================= Tool Message =================================
Name: calculator

Error: TypeError("unsupported operand type(s) for ^: 'OpNode' and 'int'")
 Please fi